In [140]:
#codigo_canhero = 6457
fecha_inicio = '2021-10-25'
fecha_final = '2021-10-28'
cod_ca = 772

In [78]:
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime

import collections
from amigocloud import AmigoCloud

import collections

In [7]:
amigocloud = AmigoCloud(token='')
amigocloud

In [141]:
query_reporte = {'query': 'select * from dataset_272975 where (fecha_registro between \'{fecha_ini}\' and \'{fecha_fin}\') and cast(SPLIT_PART(canhero, \' / \', 1) as integer) = {cod}'.format(fecha_ini=fecha_inicio, fecha_fin=fecha_final, cod=cod_ca)}
reporte_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/30837/sql', query_reporte)
reporte_data = reporte_select['data']

#credito_data['insp_fech_inf'] = datetime.now().strftime('%d/%m/%Y')
#date_update = credito_data['date_update']
#credito_data['date_update'] = datetime.strptime(date_update[:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

reporte_data


#print(query_reporte)
#print('end')

[{'ubicacion': '0101000020E610000092E3F32FEEAE4FC05254EC1E8FDB30C0',
  'amigo_id': '23e107c002ce4c169e3d37c3420e809d',
  'fecha_registro': '2021-10-25 17:00:56+00:00',
  'canhero': '772 / AGRO-SERVILOG S.R.L.',
  'entera_fresca': False,
  'entera_quemada': False,
  'picada_fresca': True,
  'picada_quemada': False,
  'quema': True,
  'hilerado': False,
  'permanente': True,
  'subsolado': True,
  'fertilizado': True,
  'pre_emergente': False,
  'post_emergente': False,
  'control_de_malezas': 3,
  'observaciones': None,
  'cultivado': True,
  'responsable': 'Juan Pablo Jaldin Duran',
  'propiedad': '588 / SIND.AGRARIO LOS ANDES-PARCELA 11',
  'superficie': 32.8977564273,
  'evaluacion': 4,
  'latitud': -16.86045364,
  'longitud': -63.3607437,
  'user': 'jjaldinduran@gmail.com'}]

In [142]:
for i in reporte_data:
    query = {'query': 'select detalle, estado, s3_filename, fo.amigo_id from dataset_276804 fo inner join gallery_37796 ga on ga.source_amigo_id = fo.amigo_id where fotografias=\'{id}\''.format(id=i['amigo_id'])}
    select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/30837/sql', query)
    data = select['data']
    i['obs']=data

In [44]:
reporte_data

[{'ubicacion': '0101000020E61000002F19C748F6A24FC08CC30545983931C0',
  'amigo_id': '9d4e441b0e6347649d276b47995ccdef',
  'fecha_registro': '2021-10-21 14:34:55+00:00',
  'canhero': '6457 / JORDAN TONDELLY RODOLFO',
  'entera_fresca': False,
  'entera_quemada': True,
  'picada_fresca': False,
  'picada_quemada': False,
  'quema': True,
  'hilerado': False,
  'permanente': False,
  'subsolado': True,
  'fertilizado': True,
  'pre_emergente': False,
  'post_emergente': True,
  'control_de_malezas': 4,
  'observaciones': 'Mal manejo, pese que realiza toda las actividades necesarias para el cultivo ',
  'cultivado': True,
  'responsable': 'Rogelio Acuña Rodríguez',
  'propiedad': '1088 / EL CHAPARRAL--RODOLFO JORDAN',
  'superficie': 40.837516221121,
  'evaluacion': 4,
  'latitud': -17.22574132,
  'longitud': -63.273084,
  'user': 'ingenieriaagricola2016@gmail.com',
  'obs': [{'detalle': 'Mal manejo, pese que realiza toda las actividades necesarias para el cultivo',
    'estado': 'Malo',
  

In [65]:
import requests
import docxtpl
from docxtpl import DocxTemplate
from docx.shared import Mm

In [143]:
for insp in reporte_data:
    doc = DocxTemplate("template/template01.docx")
    lista_fotos_inline = []
    for obs in insp['obs']:
        print(obs['amigo_id'] + '  ---  ' + obs['s3_filename'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + obs['s3_filename'])
        file = open('fotos/' + obs['amigo_id']+'.jpg', "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + obs['amigo_id']+'.jpg', width=Mm(70)), 'obs': obs['detalle'], 'estado':obs['estado']})
    data = collections.namedtuple("insp", insp.keys())(*insp.values())
    context = {'insp':data, 'obs':lista_fotos_inline}
    doc.render(context)
    doc.save('_' + str(insp['canhero'].replace('/','-')) + '-' + str(insp['propiedad'].replace('/','-')) + '.docx')

e471015419614644a74aac6c322a2cff  ---  37796_20211025125930267.jpg
46e2226f3a9c48aab5841f726f496ec6  ---  37796_20211025130028755.jpg
